In [3]:
import Pkg; Pkg.add("FiniteDifferences")

   Updating registry at `~/.julia/registries/General`
######################################################################### 100.0%
  Resolving package versions...
Updating `~/.julia/environments/v1.5/Project.toml`
  [26cc04aa] + FiniteDifferences v0.11.2
No Changes to `~/.julia/environments/v1.5/Manifest.toml`


In [4]:
using DifferentialEquations, ParameterizedFunctions
using Plots
using LinearAlgebra
using FiniteDifferences
using Plots.Measures
using ForwardDiff
gr(dpi=600,size=(800,800))
theme(:vibrant)

function OtherLV!(du,u,p,t)
    x,y,z = u
    r₁, r₂, r₃, a = p
    du[1] = r₁ * (1 + a - x - a*z)*x
    du[2] = r₂ * (x - y)*y
    du[3] = r₃ * (y - z)*z
end
p = (2.,3.,4.,10.)
tmax = 24.65
u0 = [  0.6342451880657668,  1.616845160167877,  1.5516290864728968]
prob = ODEProblem(OtherLV!,u0,(0.0,tmax),p);

┌ Info: Precompiling FiniteDifferences [26cc04aa-876d-5657-8c51-4c34ba976000]
└ @ Base loading.jl:1278


In [5]:
sol = solve(prob,Vern7(),reltol=1e-6,saveat=0.:0.01:tmax);

In [6]:
function tangent(t)
    u = sol(t)
    du = zero(u)
    OtherLV!(du,u,p,0)
    return du/norm(du)
end
T = t-> tangent(t)

function normal(t)
    N = central_fdm(5, 1)(T, t)
    return N/norm(N)
end

binormal = t -> cross(normal(t),tangent(t));

In [7]:
function rectangle(t;σ=.25)
    ut = sol(t)
    n = normal(t)
    bn = binormal(t)
    L1 = ut + σ*n +σ*bn
    L2 = ut + σ*n - σ*bn
    L3 = ut - σ*n - σ*bn
    L4 = ut - σ*n + σ*bn
    return hcat(L1,L2,L3,L4,L1)
end

rectangle (generic function with 1 method)

In [8]:
function generate3dcone(t;σ=0.1,nsamp=30)
    k = t->tangent(t);
    v = t->normal(t);
    kv = t->binormal(t);

    vrot = (t,θ)-> v(t)*cos(θ) + kv(t)*sin(θ)+k(t)*(k(t)'*v(t))*(1-cos(θ));
    pts = Array{Float64}(undef,3)
    for i in range(0., σ, length=nsamp)
        for θ in range(0., 2π, length=nsamp)
        pts = hcat(pts, (sol(t)+(1-i+σ)*tangent(t)+i*vrot(t,θ)))
        end
    end
    return pts[1,2:end], pts[2,2:end], pts[3,2:end]
end

generate3dcone (generic function with 1 method)

In [9]:
function floquetflow(t; plane_σ=1.0/sqrt(2))
    plot_phase_space=plot(sol, vars=(1,2,3),color="black")
    plot!([sol(t)[1]],[sol(t)[2]],[sol(t)[3]],st=:scatter)
    a,b,c = eachrow(rectangle(t, σ=plane_σ))
    plot!(a,b,c, lims=(-1.,4.), aspect_ratio=:equal, legend=false, ticks=0.:2.:4., color="deepskyblue2")
    plot!(
    [sol(t)[1],sol(t)[1]+tangent(t)[1]
        ],
    [sol(t)[2],sol(t)[2]+tangent(t)[2]
        ],
    [sol(t)[3],sol(t)[3]+tangent(t)[3]],color="deepskyblue2")
    plot!(generate3dcone(t),color="deepskyblue2")
    
    plot_x = plot(sol,vars=(1),color="black",ylims=(0.,3.2),legend=false)
    plot!([t],[sol(t)[1]],marker=:dot, markerstrokewidth = 3)
    xaxis!("")

    plot_y = plot(sol,vars=(2),color="black",ylims=(0.,3.2),legend=false)
    plot!([t],[sol(t)[2]],marker=:dot, markerstrokewidth = 3)
    xaxis!("")
    
    plot_z = plot(sol,vars=(3),color="black",ylims=(0.,3.2),legend=false)
    plot!([t],[sol(t)[3]],marker=:dot, markerstrokewidth = 3)
    xaxis!("")
    
    plot_out=plot(plot_x,plot_phase_space, plot_y, plot_z, layout =grid(2,2),size=(1200,1200))

end

floquetflow (generic function with 1 method)

In [10]:
floquetflow(1.2)

UndefVarError: UndefVarError: minorticks not defined

# Animate flow figure

In [7]:
dt = 0.01
anim = @animate for t in range(0.,tmax,step=dt)
    floquetflow(t)
end

┌ Warning: Keyword argument markerstrokestyle not supported with Plots.GRBackend().  Choose from: Set(Symbol[:top_margin, :group, :background_color, :yforeground_color_text, :yguidefontcolor, :seriesalpha, :legendfontcolor, :seriescolor, :ztick_direction, :zlims, :overwrite_figure, :xguidefonthalign, :normalize, :linestyle, :xflip, :fillcolor, :ygrid, :background_color_inside, :zguidefonthalign, :bins, :yscale, :xtickfontcolor, :xguide, :fillalpha, :tick_direction, :yguidefontsize, :legendfontfamily, :foreground_color, :xtickfonthalign, :x, :ytickfontrotation, :legend, :discrete_values, :ytick_direction, :xguidefontrotation, :ribbon, :tickfontrotation, :xdiscrete_values, :legendtitle, :xgridstyle, :orientation, :gridstyle, :markersize, :camera, :xforeground_color_grid, :quiver, :zticks, :markerstrokecolor, :ztickfontrotation, :ztickfonthalign, :legendfonthalign, :xtickfontsize, :levels, :zgridstyle, :foreground_color_border, :zguidefontvalign, :marker_z, :markerstrokealpha, :markeralph

Animation("C:\\Users\\piotr\\AppData\\Local\\Temp\\jl_v10260", ["000001.png", "000002.png", "000003.png", "000004.png", "000005.png", "000006.png", "000007.png", "000008.png", "000009.png", "000010.png"  …  "002457.png", "002458.png", "002459.png", "002460.png", "002461.png", "002462.png", "002463.png", "002464.png", "002465.png", "002466.png"])

In [ ]:
gif(anim, joinpath(pwd(),"anime.gif"), fps = 30)

# Poincare section

In [8]:
using DynamicalSystems

┌ Info: Precompiling DynamicalSystems [61744808-ddfa-5f27-97ff-6e42cc95d634]
└ @ Base loading.jl:1273



Update message: ChaosTools v1.9

A method to calculate the expansion entropy for discrete and
continuous systems is now included as `expansionentropy`!

See B. Hunt & E. Ott, ‘Defining Chaos’, Chaos 25.9 (2015).




In [9]:
plane = vcat(tangent(sol.t[end]),tangent(sol.t[end])'*sol.u[end]);
u0₂= [1.,2.,3.]
ds = ContinuousDynamicalSystem(OtherLV!, u0₂, p);
psos = poincaresos(ds, plane,tmax);

In [10]:
t = tmax

24.65

In [11]:
function plot_rotated_attractor(;i=10)
#    i=10
    p2=plot(sol, vars=(1,2,3),color="black",label="Attractor")
    prob₂ = ODEProblem(OtherLV!,u0₂,(0.0,1e3),p);
    sol₂ = solve(prob₂,Vern7(),reltol=1e-6,saveat=0.:0.01:tmax);
    plot!([sol(t)[1]],[sol(t)[2]],[sol(t)[3]],st=:scatter, label=())
    rect = rectangle(t,σ=1.5)
    a,b,c = eachrow(rect)
    plot!(a,b,c)
    tangent(tmax)
    n = length(psos)
    plot!(columns(psos),st=:scatter, marker_z=n:-1:1,color=:kr,colorbar=false)
    plot!(sol₂(0.:0.01:tmax)[1,:],sol₂(0.:0.01:tmax)[2,:],sol₂(0.:0.01:tmax)[3,:], xlims=(-1.0, 5), ylims=(-1.0,5.0), zlims=(-1.0,5.0),ls=:dash,leg=false, aspect_ratio=:equal, ticks=(0.:2.:4.),
    line_z=tmax:-0.01:0.,color=:kr,colorbar=false, camera = (2*i,5*i),size=(1200,1200))
    return p2
end

plot_rotated_attractor (generic function with 1 method)

In [38]:
anim = @animate for i in range(0.,10.,step=0.05)
    plot_rotated_attractor(;i=i)
end
mp4(anim, joinpath(pwd(),"rotation.mp4"), fps = 120)

┌ Info: Saved animation to 
│   fn = C:\Users\piotr\Documents\Julia\limitcycle\experiments\rotation.mp4
└ @ Plots C:\Users\piotr\.julia\packages\Plots\qZHsp\src\animation.jl:98


Plots.AnimatedGif("C:\\Users\\piotr\\Documents\\Julia\\limitcycle\\experiments\\rotation.mp4")

In [12]:
function projection(t)
    P = -hcat(normal(t),binormal(t))
end

projection (generic function with 1 method)

In [13]:
A = projection(tmax)
planar_coords = zeros(length(psos),2,);
for i in 1:length(psos)
    planar_coords[i,:]= psos[i]'*A
end
n = length(psos);

In [17]:
rect = rectangle(t,σ=1.5)
minax= mapslices(u->min(u...), rect'*A, dims=1)
maxax= mapslices(u->max(u...), rect'*A, dims=1)
ylims=[minax[2],maxax[2]]
println(ylims)

xlims=[minax[1],maxax[1]]
println(xlims)

[-0.7601135854406637, 2.2398864145593373]
[0.22949569272856649, 3.229495692728567]


In [18]:
i = 10

10

In [19]:
    pproj = plot([planar_coords[1:i,1]],[planar_coords[1:i,2]], st=:scatter, legend=false, xlims=[minax[1],maxax[1]],
            ylims=[minax[2],maxax[2]],aspect_ratio=:equal, marker_z=n:-1:1,color=:kr)
    p2 = plot_rotated_attractor()
    plot(p2,pproj,size=(1200,600))
savefig("poincare_section.png")

In [99]:
poincare_anim = @animate for i in repeat(1:length(psos),inner=3)

end
gif(poincare_anim, joinpath(pwd(),"poincare_anim.gif"), fps = 6);

┌ Info: Saved animation to 
│   fn = C:\Users\piotr\Documents\Julia\limitcycle\experiments\poincare_anim.gif
└ @ Plots C:\Users\piotr\.julia\packages\Plots\qZHsp\src\animation.jl:98
